In [117]:
import numpy as np
import re, os, pygtrie, numbers
dimension = 30
def readFile(path):
    list = []
    input = open(path, 'r')
    list.append(input.readlines())
    return(list)
def process_line(string):
    string = re.sub(r'[^A-Za-z0-9\ \.]', "", string)
    string = re.sub('[0-9]', "0",string)
    string = string.lower()
    string = "##" + string + "#"
    return(string)
processed_lines = []
list = readFile("assignment1-data/training.en")
for l in list:
    for line in l:
        processed_lines.append(process_line(line))
#     print('-------------------')
# print(processed_lines)
# for processed_line in processed_lines:
print(processed_lines[0])

triGram = np.zeros((30,30,30))
# print(processed_lines)
for processed_line in processed_lines:
    constructTriGram(processed_line.strip(' ')) # shift it from the for above
print('done')

##resumption of the session#
done


In [118]:
def constructTriGram(line):
    length = len(line)
    listOfGrams = []
    for i in range(length):
        c = []
        if i is 0 or i is 1:
            continue
        else:
            enterSequence(line[i-2:i+1])

In [286]:
ord('.')%97

46

In [116]:
def enterSequence(sequence):
    seq=[]
    seq.append(ord(sequence[0])%97)
    seq.append(ord(sequence[1])%97)
    seq.append(ord(sequence[2])%97)
    seq = getascii(seq)
    i = seq[0]
    j = seq[1]
    k = seq[2]
    if ((i == 29 and j == 29 and k == 29)):
        print('###')
        print(asciiToChars(i)+asciiToChars(j)+asciiToChars(k))
    elif (j == 29 and k == 29):
        print('c##')
        print(asciiToChars(i)+asciiToChars(j)+asciiToChars(k))
    elif (i == 29 and k == 29):
        print('#c#')
        print(asciiToChars(i)+asciiToChars(j)+asciiToChars(k))
    elif (i == 29 and k == 29 and j!=29):
        print('#c#')
        print(asciiToChars(i)+asciiToChars(j)+asciiToChars(k))
    elif (i != 29 and j == 29 and k!=29):
        print('c#c')
        print(asciiToChars(i)+asciiToChars(j)+asciiToChars(k))
    else:
        triGram[i][j][k] += 1

In [31]:
def getascii(seq):
    for i in range(len(seq)):
        if seq[i] == 32:
            seq[i] = 26
        if seq[i] == 46:
            seq[i] = 27
        if seq[i] == 48:
            seq[i] = 28
        if seq[i] == 35:
            seq[i] = 29
    return seq

In [72]:
def getasciiForSingleChar(character):
    char = ord(character)%97
    if char == 32:
        char = 26
    if char == 46:
        char = 27
    if char == 48:
        char = 28
    if char == 35:
        char = 29
    return char

In [33]:
def getBigram(freq_matrix):
    bi_grams = np.zeros((dimension,dimension))
    for i in range(dimension):
        for j in range(dimension):
            bi_grams[i][j] = sum(freq_matrix[i][j])
    return(bi_grams)

In [34]:
def getUnigram(bi_grams):
    uni_grams = np.zeros((dimension))
    for i in range(dimension):
        uni_grams[i] = sum(bi_grams[i])
    return(uni_grams)

In [119]:
def generateKNTriGram():
    # D = [0.70, 0.20, 0.10]
    D = 0.75
    charCounts = dimension
    bi_gram_count = getBigram(triGram)
    uni_gram_count = getUnigram(bi_gram_count)
    uni_gram = (uni_gram_count / sum(uni_gram_count))
    bi_gram = np.zeros((dimension,dimension))
    tri_gram = np.zeros((dimension,dimension,dimension))
    for i in range(dimension):
        for j in range(dimension):
            if bi_gram_count[i][j] > 0:
                bi_gram[i][j] = ( (bi_gram_count[i][j] - D) / uni_gram_count[i] ) + (((D * charCounts) / uni_gram_count[i]) * uni_gram[j])
            else:
                bi_gram[i][j] = (((D * charCounts) / uni_gram_count[i]) * uni_gram[j])
                # print(bi_gram)
    for w1 in range(dimension):
        for w2 in range(dimension):
            for w3 in range(dimension):
                if triGram[w1][w2][w3] > 0:
                    tri_gram[w1][w2][w3] = ((triGram[w1][w2][w3] - D) / bi_gram_count[w1][w2]) + ((D * charCounts) / bi_gram_count[w1][w2]) * bi_gram[w2][w3]
                elif bi_gram_count[w1][w2] > 0:
                    tri_gram[w1][w2][w3] = ((D * charCounts) / bi_gram_count[w1][w2]) * bi_gram[w2][w3]
#                 else:
#                     tri_gram[w1][w2][w3] = 
                    ##################### Work in pending #####################
    print(tri_gram)


In [120]:
generateKNTriGram()       

[[[  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [  1.01543529e-02   1.85899338e-05   1.48773777e-03 ...,   1.00508228e-04
     8.26867193e-06   2.33248856e-05]
  [  6.10435399e-03   4.02414690e-06   2.55761093e-01 ...,   1.43284707e-04
     1.45019079e-04   5.04911782e-06]
  ..., 
  [  8.18656943e-02   1.39446374e-02   3.54302267e-02 ...,   1.35553423e-01
     3.82205151e-01   9.73378761e-01]
  [  5.39805530e-03   9.19480674e-04   4.29964571e-02 ...,   8.82752081e-02
     2.92281289e+00   1.15367713e-03]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]]

 [[  2.59668916e-05   1.32692173e-02   6.36429198e-02 ...,   3.10147145e-04
     1.40313463e-04   5.54966695e-06]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [  1.95339328e+00   1.28772701e-03   5.93549606e-01 ...,   4.58511062e-02
  

In [455]:
from random import randint
theText = []
def generateText():
    temp = 0
    first = 0
    second = 0
    theOne = 0
    for i in range(500):
        theOne = randint(0,2)
        probableChar = sorted(range(len(tri_gram_add_one[first][second])), key=lambda i: tri_gram_add_one[first][second][i], reverse=True)[:3]
        theText.append((probableChar[theOne]))
        temp = (probableChar[theOne])
        first = second
        second = temp
generateText()
text = ' '
for i in range(len(theText)):
    text = text + asciiToChars(theText[i])
print(text)

 ccoures isenter a fuldinted is asent to bybbcatint oneregratiod a sen one tor to astake clan ouregrourouroultunise trinted.bat iss ase clan tring.#buropmad and a posent int traters it trioduchase astrioulans ansit trallempoleast oft it offor tratint is trulds.accus isse cludirs to bused astanspole coure tor as astruct in ould in ands thas ans tor se anspan thing to thesithis ingenteres ans the can ous a sends a sent oust is thativelin on trinte clustrultuatious isenden an as isedused.bcbalst tri


In [94]:
def asciiToChars(value):
    char =''
    if value < 26:
        char = chr(int(value+97))
    elif value == 26:
        char = ' '
    elif value == 27:
        char = '.'
    elif value == 28:
        char = '0'
    elif value == 29:
        char = '#'
    return char

In [95]:
def getTrainAndTestData(corpus, dev_part):
    length = len(corpus)
    folded_data = []
    folds = 5
    fold = int(length/5)
    start = 0
    print(fold)
    for i in range(len(corpus)):
        if (i%fold == 0):
            folded_data.append(corpus[start:i])
            start = i
    print(folded_data)
    dev = folded_data[dev_part]
    train = folded_data.pop(dev_part)
    print(dev)
    print('---')
    print(train)

In [311]:
getTrainAndTestData("Here we go, I am just checking",1)

6
['', 'Here w', 'e go, ', 'I am j', 'ust ch']
Here w
---
Here w


In [317]:
from nltk.tokenize import sent_tokenize
def getStratifiedSamplingOfData(corpus, dev_part):
    sent_list = sent_tokenize(corpus)
    train_data = []
    test_data = []
    for i in range(len(sent_list)):
        if i%5 == 0:
            test_data.append(sent_list[i])
        else:
            train_data.append(sent_list[i])
    print(train_data)        
    
    

In [318]:
whichPart = 0
corpus = "this’s a sent tokenize test. this is sent two. is this sent three? sent 4 is cool! Now it’s your turn."
getStratifiedSamplingOfData(corpus, whichPart)

['this is sent two.', 'is this sent three?', 'sent 4 is cool!', 'Now it’s your turn.']


In [400]:
print(bi_gram)

[[  1.43346734e-04   1.87918407e-02   3.42502653e-02   3.30734188e-02
    4.87425055e-04   1.07768133e-02   1.71923935e-02   2.23787232e-04
    2.75960204e-02   1.74324081e-03   1.86698303e-02   1.22287842e-01
    4.13140766e-02   1.82647944e-01   1.85961195e-04   1.77477963e-02
    2.33333729e-06   1.10239081e-01   8.41098725e-02   1.64491146e-01
    1.11134935e-02   2.43566011e-02   4.34093918e-03   1.63702650e-03
    2.14725274e-02   2.74477679e-05   5.05311524e-02   3.22368968e-07
    7.87004990e-04]
 [  7.95154346e-02   1.43346734e-04   5.20714485e-04   3.70342411e-04
    4.19944457e-01   2.41132258e-04   1.83775556e-04   5.27465559e-04
    2.99570591e-02   3.84914464e-02   5.58259895e-05   1.28721026e-01
    4.89374826e-03   8.26944980e-04   4.82013226e-02   3.00830050e-04
    1.36863716e-05   3.16661696e-02   3.60665761e-02   5.66477721e-03
    9.03126901e-02   9.05742008e-04   1.79453544e-04   2.29606892e-05
    8.19789328e-02   3.96184442e-06   6.82154574e-03   1.89088029e-06


In [186]:
from decimal import Decimal
def createArpa(): 
    file = open("trigram_add_one.en","w")
    for i in range(30):
        for j in range(30):
            for k in range(30):
                if ((i == 29 and j == 29 and k == 29)):
#                     print('###')
                    print(asciiToChars(i)+asciiToChars(j)+asciiToChars(k))
                    continue
                elif (j == 29 and k == 29):
#                     print('c##')
                    print(asciiToChars(i)+asciiToChars(j)+asciiToChars(k))
                    continue
                elif (i != 29 and j == 29 and k!=29):
#                     print('c#c')
                    print(asciiToChars(i)+asciiToChars(j)+asciiToChars(k))
                    continue
                else:
                    s = ""
                    listSeq = []
                    listSeq.append(asciiToChars(i))
                    listSeq.append(asciiToChars(j))
                    listSeq.append(asciiToChars(k))
                    listSeq.append("\t")
                    listSeq.append(str(tri_gram_one[i][j][k]))
#                     print(tri_gram_one[i][j][k])
                    listSeq.append("\n")
                    file.write(s.join(listSeq)) 
    file.close()



In [187]:
createArpa()

a#a
a#b
a#c
a#d
a#e
a#f
a#g
a#h
a#i
a#j
a#k
a#l
a#m
a#n
a#o
a#p
a#q
a#r
a#s
a#t
a#u
a#v
a#w
a#x
a#y
a#z
a# 
a#.
a#0
a##
b#a
b#b
b#c
b#d
b#e
b#f
b#g
b#h
b#i
b#j
b#k
b#l
b#m
b#n
b#o
b#p
b#q
b#r
b#s
b#t
b#u
b#v
b#w
b#x
b#y
b#z
b# 
b#.
b#0
b##
c#a
c#b
c#c
c#d
c#e
c#f
c#g
c#h
c#i
c#j
c#k
c#l
c#m
c#n
c#o
c#p
c#q
c#r
c#s
c#t
c#u
c#v
c#w
c#x
c#y
c#z
c# 
c#.
c#0
c##
d#a
d#b
d#c
d#d
d#e
d#f
d#g
d#h
d#i
d#j
d#k
d#l
d#m
d#n
d#o
d#p
d#q
d#r
d#s
d#t
d#u
d#v
d#w
d#x
d#y
d#z
d# 
d#.
d#0
d##
e#a
e#b
e#c
e#d
e#e
e#f
e#g
e#h
e#i
e#j
e#k
e#l
e#m
e#n
e#o
e#p
e#q
e#r
e#s
e#t
e#u
e#v
e#w
e#x
e#y
e#z
e# 
e#.
e#0
e##
f#a
f#b
f#c
f#d
f#e
f#f
f#g
f#h
f#i
f#j
f#k
f#l
f#m
f#n
f#o
f#p
f#q
f#r
f#s
f#t
f#u
f#v
f#w
f#x
f#y
f#z
f# 
f#.
f#0
f##
g#a
g#b
g#c
g#d
g#e
g#f
g#g
g#h
g#i
g#j
g#k
g#l
g#m
g#n
g#o
g#p
g#q
g#r
g#s
g#t
g#u
g#v
g#w
g#x
g#y
g#z
g# 
g#.
g#0
g##
h#a
h#b
h#c
h#d
h#e
h#f
h#g
h#h
h#i
h#j
h#k
h#l
h#m
h#n
h#o
h#p
h#q
h#r
h#s
h#t
h#u
h#v
h#w
h#x
h#y
h#z
h# 
h#.
h#0
h##
i#a
i#b
i#c
i#d
i#e
i#f
i#g
i#h
i#i
i#j


In [183]:
def generateAddOneSmoothing(alpha):
    tri_gram_add_one = np.zeros((dimension, dimension, dimension))
    for w1 in range(dimension):
        for w2 in range(dimension):
            for w3 in range(dimension):
                tri_gram_add_one[w1][w2][w3] = ((triGram[w1][w2][w3] + alpha) / (sum(triGram[w1][w2]) + (alpha*(dimension))))
    return(tri_gram_add_one)

In [184]:
tri_gram_one = generateAddOneSmoothing(1)

In [21]:
from __future__ import print_function
import numpy as np

# method for generating text
def generate_text(model, length, vocab_size, ix_to_char):
    ix = [np.random.randint(vocab_size)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, vocab_size))
    for i in range(length):
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)


def load_data(data_dir, seq_length):
    data = open(data_dir, 'r').read()
    chars = list(set(data))
    VOCAB_SIZE = len(chars)

    print('Data length: {} characters'.format(len(data)))
    print('Vocabulary size: {} characters'.format(VOCAB_SIZE))

    ix_to_char = {ix:char for ix, char in enumerate(chars)}
    char_to_ix = {char:ix for ix, char in enumerate(chars)}

    X = np.zeros((int(len(data)/seq_length), seq_length, VOCAB_SIZE))
    y = np.zeros((int(len(data)/seq_length), seq_length, VOCAB_SIZE))
    for i in range(0, int(len(data)/seq_length)):
        X_sequence = data[i*seq_length:(i+1)*seq_length]
        X_sequence_ix = [char_to_ix[value] for value in X_sequence]
        input_sequence = np.zeros((seq_length, VOCAB_SIZE))
        for j in range(seq_length):
            input_sequence[j][X_sequence_ix[j]] = 1.
            X[i] = input_sequence

        y_sequence = data[i*seq_length+1:(i+1)*seq_length+1]
        y_sequence_ix = [char_to_ix[value] for value in y_sequence]
        target_sequence = np.zeros((seq_length, VOCAB_SIZE))
        for j in range(seq_length):
            target_sequence[j][y_sequence_ix[j]] = 1.
            y[i] = target_sequence
    return X, y, VOCAB_SIZE, ix_to_char


In [26]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import time
import csv
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers.wrappers import TimeDistributed

BATCH_SIZE = 50
HIDDEN_DIM = 500
SEQ_LENGTH = 50
WEIGHTS = ''
mode = 'train'
GENERATE_LENGTH = 500
LAYER_NUM = 2

X, y, VOCAB_SIZE, ix_to_char = load_data('assignment1-data/training.en', SEQ_LENGTH)
model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LAYER_NUM - 1):
  model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")
generate_text(model, 500, VOCAB_SIZE, ix_to_char)
if not WEIGHTS == '':
  model.load_weights(WEIGHTS)
  nb_epoch = int(WEIGHTS[WEIGHTS.rfind('_') + 1:WEIGHTS.find('.')])
else:
  nb_epoch = 0
if mode == 'train' or WEIGHTS == '':
  while nb_epoch<33:
    print('\n\nEpoch: {}\n'.format(nb_epoch))
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, nb_epoch=1)
    nb_epoch += 1
    generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, ix_to_char)
    if nb_epoch % 10 == 0:
      model.save_weights('checkpoint_layer_{}_hidden_{}_epoch_{}.hdf5'.format(LAYER_NUM, HIDDEN_DIM, nb_epoch))

elif WEIGHTS == '':
  model.load_weights(WEIGHTS)
  generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, ix_to_char)
  print('\n\n')
else:
  print('\n\nNothing to do!')

Data length: 156902 characters
Vocabulary size: 85 characters
]óVbb'DFF'qqSwwwo!!!!D!!(qqqqwwNNN(((SDD!((55D!!D(((''QJJqqqzzJ777777777º[[4-----VV6666KKK666KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66KK66K6K6K66

Epoch: 0



/anaconda/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
3138/3138 [==============================] - 83s - loss: 3.1801    
D                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

Epoch: 1

Epoch 1/1
3138/3138 [==============================] - 84s - loss: 2.7785    

o the the tor to the tor to the tor to the tor to the tor to the tor to the tor to the tor to the tor to the tor to the tor to the tor to the tor to the tor to the tor to the tor to the tor to the tor to the tor to the tor to the tor to the tor to the tor to the tor to the tor to the tor to the tor to the tor to the tor to the to

KeyboardInterrupt: 

In [4]:
%tb

SystemExit: 2

In [82]:
def perplexity(lines, model=''):
    doc_length = 0
    total_log_sum = 0
    line_perplexities = []
    for line in lines:
        line_perplexity = 0
        for char_index in range(len(line) - 2):
#             print(ord(line[char_index]))
            probability = tri_gram_add_one[getasciiForSingleChar(line[char_index])][getasciiForSingleChar(line[char_index+1])][getasciiForSingleChar(line[char_index+2])]
            if probability > 0:
                probability = np.log2(probability)
            line_perplexity += probability
        line_perplexities.append(line_perplexity)
        doc_length += len(line)
    for line_perplexity in line_perplexities:
        total_log_sum += line_perplexity
    perplexity = 2 ** -((1/(total_log_sum-2))*doc_length)
    print(perplexity)
    
    

#     perplexity = 2 ** -((1/(sum3-2))*sum2)
#     print perplexity
#     return perplexity

In [91]:
processed_test_lines = []
list = readFile("test")
for l in list:
    for line in l:
        processed_test_lines.append(process_line(line))
perplexity(processed_test_lines)


1.1735456014
